In [8]:
from IPython.core.interactiveshell import InteractiveShell
from IPython.display import Markdown as md
import pandas as pd

InteractiveShell.ast_node_interactivity = "all"

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [48]:
from statsmodels.formula.api import ols
from scipy.stats import pearsonr
import numpy as np

In [9]:
regression_df = pd.read_excel(io='regression_data.xlsx', sheet_name=0)
regression_df.copy().iloc[:5]

,CASE,PRO_OR_CON,PERIOD,TICKER,U,U_0,LEAK,VTO,MV,RET,VOL,COR,DPRO,DCON
0,PRO-1,PRO,1,ESSA,-0.002226,0.021033,-0.007838,-0.710216,4.720191e+12,-0.010857,0.000621,1.0,1,0
1,PRO-1,PRO,2,ESSA,-0.015557,0.021033,-0.007838,0.307342,4.616829e+12,-0.010857,0.000621,1.0,1,0
2,PRO-1,PRO,3,ESSA,0.013312,0.021033,-0.007838,0.678265,4.720191e+12,-0.010857,0.000621,1.0,1,0
3,PRO-1,PRO,4,ESSA,-0.020342,0.021033,-0.007838,-0.423760,4.616829e+12,-0.010857,0.000621,1.0,1,0
4,PRO-1,PRO,5,ESSA,-0.004646,0.021033,-0.007838,-0.134473,4.616829e+12,-0.010857,0.000621,1.0,1,0


In [49]:
formula_without_dummy = 'U ~ U_0 + LEAK + VTO + np.log(MV) + RET + VOL + COR'
selected_variable = ['U_0', 'LEAK', 'VTO', 'MV', 'RET', 'VOL', 'COR']

In [26]:
data_pro = regression_df[regression_df['PRO_OR_CON'] == 'PRO']
model1 = ols(formula=formula_without_dummy, data=data_pro).fit()

In [28]:
model1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      U   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     5.089
Date:                Tue, 05 Sep 2023   Prob (F-statistic):           8.59e-06
Time:                        19:09:50   Log-Likelihood:                 66286.
No. Observations:               34000   AIC:                        -1.326e+05
Df Residuals:                   33992   BIC:                        -1.325e+05
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0071      0.003     -2.405      0.016      -0.013      -0.001
U_0           -0.0138      0.006     -2.221      0.026      -0.026      -0.002
LEAK         7.72e-05      0.004      0.017      0.986      -0.009       0.009
VTO         1.299e-09   1.15e-09      1.131      0.258   -9.53e-10    3.55e-09
np.log(MV)     0.0002   9.94e-05      2.311      0.021    3.49e-05       0.000
RET            0.0730      0.028      2.648      0.008       0.019       0.127
VOL            0.2883      0.096      3.006      0.003       0.100       0.476
COR            0.0002      0.001      0.279      0.780      -0.001       0.002
==============================================================================
Omnibus:                    14915.089   Durbin-Watson:                   2.113
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           688684.427
Skew:                           1.380   Prob(JB):                         0.00
Kurtosis:                      24.875   Cond. No.                     8.37e+07
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.37e+07. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [56]:
corr1 = data_pro.copy()[selected_variable].corr()
pearson1 = data_pro.copy()[selected_variable].corr(method=lambda x, y: pearsonr(x, y)[1]) - np.eye(*corr1.shape)
symbol1 = pearson1.map(lambda x: ''.join(['*' for t in [.05, .01, .001] if x<=t]))
corr1.round(6).astype(str) + symbol1

,U_0,LEAK,VTO,MV,RET,VOL,COR
U_0,1.0***,-0.17069***,-0.000606,0.040665***,-0.124441***,0.011135*,0.0276***
LEAK,-0.17069***,1.0***,0.000593,0.065559***,0.490657***,0.089782***,0.07867***
VTO,-0.000606,0.000593,1.0***,-0.001768,0.003706,0.00031,0.007306
MV,0.040665***,0.065559***,-0.001768,1.0***,0.035116***,-0.04468***,-0.084538***
RET,-0.124441***,0.490657***,0.003706,0.035116***,1.0***,0.221088***,0.010875*
VOL,0.011135*,0.089782***,0.00031,-0.04468***,0.221088***,1.0***,0.019818***
COR,0.0276***,0.07867***,0.007306,-0.084538***,0.010875*,0.019818***,1.0***


In [32]:
data_con = regression_df[regression_df['PRO_OR_CON'] == 'CON']
model2 = ols(formula=formula_without_dummy, data=data_con).fit()

In [33]:
model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      U   R-squared:                       0.008
Model:                            OLS   Adj. R-squared:                  0.007
Method:                 Least Squares   F-statistic:                     37.03
Date:                Tue, 05 Sep 2023   Prob (F-statistic):           4.84e-52
Time:                        19:12:16   Log-Likelihood:                 46366.
No. Observations:               34000   AIC:                        -9.272e+04
Df Residuals:                   33992   BIC:                        -9.265e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0092      0.005     -1.765      0.078      -0.019       0.001
U_0           -0.1238      0.008    -15.820      0.000      -0.139      -0.108
LEAK          -0.0004      0.006     -0.064      0.949      -0.012       0.011
VTO        -1.114e-10   6.75e-10     -0.165      0.869   -1.43e-09    1.21e-09
np.log(MV)     0.0003      0.000      1.895      0.058   -1.14e-05       0.001
RET            0.0045      0.036      0.124      0.901      -0.066       0.075
VOL           -0.2036      0.116     -1.758      0.079      -0.431       0.023
COR            0.0006      0.002      0.412      0.680      -0.002       0.004
==============================================================================
Omnibus:                   133282.417   Durbin-Watson:                   2.024
Prob(Omnibus):                  0.000   Jarque-Bera (JB):     281143097378.484
Skew:                          95.638   Prob(JB):                         0.00
Kurtosis:                   14089.070   Cond. No.                     1.73e+08
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.73e+08. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [57]:
corr2 = data_con.copy()[selected_variable].corr()
pearson2 = data_con.copy()[selected_variable].corr(method=lambda x, y: pearsonr(x, y)[1]) - np.eye(*corr2.shape)
symbol2 = pearson2.map(lambda x: ''.join(['*' for t in [.05, .01, .001] if x<=t]))
corr2.round(6).astype(str) + symbol2

,U_0,LEAK,VTO,MV,RET,VOL,COR
U_0,1.0***,0.001552,0.028684***,0.003004,0.065066***,-0.071269***,-0.025723***
LEAK,0.001552,1.0***,0.026439***,-0.000551,0.547941***,0.38572***,-0.014802**
VTO,0.028684***,0.026439***,1.0***,-0.002141,0.01309*,0.012611*,-0.001799
MV,0.003004,-0.000551,-0.002141,1.0***,0.020392***,-0.057002***,-0.100394***
RET,0.065066***,0.547941***,0.01309*,0.020392***,1.0***,0.430726***,-0.028598***
VOL,-0.071269***,0.38572***,0.012611*,-0.057002***,0.430726***,1.0***,-0.0424***
COR,-0.025723***,-0.014802**,-0.001799,-0.100394***,-0.028598***,-0.0424***,1.0***


In [36]:
formula_with_dummy = 'U ~ U_0 + LEAK + VTO + np.log(MV) + RET + VOL + COR + DPRO + DCON'

In [39]:
model3 = ols(formula=formula_with_dummy, data=regression_df).fit()

In [47]:
model3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      U   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:                     37.52
Date:                Tue, 05 Sep 2023   Prob (F-statistic):           5.15e-60
Time:                        19:32:41   Log-Likelihood:             1.0707e+05
No. Observations:               68000   AIC:                        -2.141e+05
Df Residuals:                   67991   BIC:                        -2.140e+05
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0063      0.002     -3.154      0.002      -0.010      -0.002
U_0           -0.0876      0.005    -17.018      0.000      -0.098      -0.078
LEAK          -0.0035      0.004     -0.930      0.352      -0.011       0.004
VTO        -6.133e-11    5.2e-10     -0.118      0.906   -1.08e-09    9.57e-10
np.log(MV)     0.0003      0.000      3.096      0.002       0.000       0.001
RET            0.0085      0.023      0.362      0.717      -0.037       0.054
VOL           -0.0109      0.077     -0.143      0.886      -0.161       0.139
COR            0.0008      0.001      0.939      0.348      -0.001       0.003
DPRO          -0.0036      0.001     -3.494      0.000      -0.006      -0.002
DCON          -0.0028      0.001     -2.699      0.007      -0.005      -0.001
==============================================================================
Omnibus:                   262705.606   Durbin-Watson:                   2.044
Prob(Omnibus):                  0.000   Jarque-Bera (JB):     781291510059.668
Skew:                          91.190   Prob(JB):                         0.00
Kurtosis:                   16607.719   Cond. No.                     6.66e+19
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 2.1e-24. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [58]:
corr3 = data_con.copy()[selected_variable].corr()
pearson3 = data_con.copy()[selected_variable].corr(method=lambda x, y: pearsonr(x, y)[1]) - np.eye(*corr3.shape)
symbol3 = pearson3.map(lambda x: ''.join(['*' for t in [.05, .01, .001] if x<=t]))
corr3.round(6).astype(str) + symbol2

,U_0,LEAK,VTO,MV,RET,VOL,COR
U_0,1.0***,0.001552,0.028684***,0.003004,0.065066***,-0.071269***,-0.025723***
LEAK,0.001552,1.0***,0.026439***,-0.000551,0.547941***,0.38572***,-0.014802**
VTO,0.028684***,0.026439***,1.0***,-0.002141,0.01309*,0.012611*,-0.001799
MV,0.003004,-0.000551,-0.002141,1.0***,0.020392***,-0.057002***,-0.100394***
RET,0.065066***,0.547941***,0.01309*,0.020392***,1.0***,0.430726***,-0.028598***
VOL,-0.071269***,0.38572***,0.012611*,-0.057002***,0.430726***,1.0***,-0.0424***
COR,-0.025723***,-0.014802**,-0.001799,-0.100394***,-0.028598***,-0.0424***,1.0***
